In [48]:
import zipfile
import hashlib
import csv
import os
import io

In [6]:
# jar 파일을 압축을 해지합니다.
jarPaths = []
password = b"b002cd031989a8118aab022d74616efd5718f399"

for i in range(2):
    inputDir = input("jar 파일 경로와 이름을 입력하세요. : ")
    jarPaths.append(inputDir)

jar 파일 경로와 이름을 입력하세요. : D:\00_Script\자동화\mars_server_v2.5.5.13.20.slsp
jar 파일 경로와 이름을 입력하세요. : D:\00_Script\자동화\mars_server_v2.5.5.13.21.slsp


In [120]:
# 파일의 해시값을 계산합니다.
def compute_hash(file, algorithm="sha256"):
    hash_func = hashlib.new(algorithm)
    hash_func.update(file.read())
    return hash_func.hexdigest()

In [121]:
# 압축 파일내에 HASH 값을 계산하고 CSV 파일로 저장합니다. 
def get_file_hashes(jar_path):
    file_hashes = {}
    filePath, fileName = os.path.split(jar_path)
        
    with open(filePath + "\\" + fileName + ".csv", mode = "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['jar File', 'File Name', 'Hash Value'])
        
        with zipfile.ZipFile(jar_path, 'r') as package:
            for file_name in package.namelist():
                if not file_name.endswith("/"):  # Skip directories
                    with package.open(file_name, pwd=password) as file:
                        # Compute and store the hash
                        #print(file_name)
                        file_hashes[file_name] = compute_hash(file)
                        writer.writerow([jarPaths[i], file_name, file_hashes[file_name]])
                                
    return file_hashes

In [122]:
# Get file hashes from both .jar files
hashes_jar1 = get_file_hashes(jarPaths[0])
hashes_jar2 = get_file_hashes(jarPaths[1])

In [24]:
outputCsv = input("결과 csv 저장할 경로와 파일명을 입력하세요.")

결과 csv 저장할 경로와 파일명을 입력하세요.D:\00_Script\자동화\test.csv


In [25]:
# Compare the hashes of files in both .jar files
with open(outputCsv, "w", newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['file_name', jarPaths[0], jarPaths[1]])
    
    for file_name in hashes_jar1:
        if file_name in hashes_jar2:
            if hashes_jar1[file_name] != hashes_jar2[file_name]:
                writer.writerow([file_name, hashes_jar1[file_name], hashes_jar2[file_name]])
                # Print file name and hash values if they differ
                print(f"File '{file_name}' differs between the jars.")
                print(f"  Hash in {jarPaths[0]}: {hashes_jar1[file_name]}")
                print(f"  Hash in {jarPaths[1]}: {hashes_jar2[file_name]}")
        else:
            # Print file name and hash value if it's only in the first jar
            writer.writerow([file_name, hashes_jar1[file_name], "-"])
            print(f"File '{file_name}' is only present in {jarPaths[0]}.")
            print(f"  Hash: {hashes_jar1[file_name]}")

    # Check for files in the second jar that are not in the first jar
    for file_name in hashes_jar2:
        if file_name not in hashes_jar1:
            # Print file name and hash value if it's only in the second jar
            writer.writerow([file_name, "-", hashes_jar2[file_name]])
            print(f"File '{file_name}' is only present in {jarPaths[1]}.")
            print(f"  Hash: {hashes_jar2[file_name]}")


File 'BALTHAZAR/lib/cdr-5.0.0.5.jar' is only present in D:\00_Script\자동화\mars_server_v2.5.5.13.20.slsp.
  Hash: fb8bbb4106bc2b101d2cb8000f107874e9239f392f2090d007a102193461bd4b
File 'BALTHAZAR/balthazar-2.5.5.13.12.jar' is only present in D:\00_Script\자동화\mars_server_v2.5.5.13.20.slsp.
  Hash: 59cf8a8e6b46f917100e754bcb55e290aa290d16dce306c8caf7e2772d096283
File 'MELCHIOR/lib/melchior-2.5.5.13.15.jar' is only present in D:\00_Script\자동화\mars_server_v2.5.5.13.20.slsp.
  Hash: c6b3cb19b4e15a746b81bd52f40880a264f3ce237e1f94536bfc64f1187b14c3
File 'WEB-CONSOLE/web-console.version' differs between the jars.
  Hash in D:\00_Script\자동화\mars_server_v2.5.5.13.20.slsp: 36e08ce01b1b826b4ded34d6f9887f4b8e1c9b8840816c794f278908c0889c57
  Hash in D:\00_Script\자동화\mars_server_v2.5.5.13.21.slsp: edbe72499b080c4f3522de0e6fe6594b0f225ad188c8fb412b178a462cd112b7
File 'WEB-CONSOLE/web-console' differs between the jars.
  Hash in D:\00_Script\자동화\mars_server_v2.5.5.13.20.slsp: af1773dde1de7ee604b4e8f454f14

In [113]:
# General function to extract hashes from nested archives (.zip or .jar)
def extract_hashes_from_nested_archive(archive_data):
    nested_file_hashes = {}

    # If archive_data is a BytesIO object, use .getvalue() to extract raw bytes
    if isinstance(archive_data, io.BytesIO):
        archive_data = archive_data.getvalue()

    # Open the archive (can be .zip or .jar) from raw bytes
    with zipfile.ZipFile(io.BytesIO(archive_data)) as archive:
        for file_name in archive.namelist():
            # Skip directories
            if file_name.endswith("/"):
                continue
            
            # Skip META-INF/MANIFEST.MF if it's a .jar file
            if file_name == "META-INF/MANIFEST.MF":
                continue

            # Check if the file itself is a .jar (or another nested archive)
            if file_name.endswith(".jar"):
                with archive.open(file_name) as nested_jar_file:
                    # Recursively extract hashes from the nested .jar
                    nested_data = io.BytesIO(nested_jar_file.read())
                    nested_hashes = extract_hashes_from_nested_archive(nested_data)
                    for nested_name, nested_hash in nested_hashes.items():
                        nested_file_hashes[f"{file_name}/{nested_name}"] = nested_hash
            else:
                # For regular files, compute the hash
                with archive.open(file_name) as file:
                    nested_file_hashes[file_name] = compute_hash(file)

    return nested_file_hashes